In [1]:
from transformers import MT5ForConditionalGeneration,T5Tokenizer
from transformers import BertTokenizer

In [2]:
import jieba
class T5PegasusTokenizer(BertTokenizer):
    def __init__(self, pre_tokenizer=lambda x: jieba.cut(x, HMM=False), *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.pre_tokenizer = pre_tokenizer

    def _tokenize(self, text, *arg, **kwargs):
        split_tokens = []
        for text in self.pre_tokenizer(text):
            if text in self.vocab:
                split_tokens.append(text)
            else:
                split_tokens.extend(super()._tokenize(text))
        return split_tokens

In [2]:
import json

In [3]:
data=json.load(open(r'C:\Users\wie\Documents\项目\文本摘要\data\sample.json','r',encoding='utf-8'))

In [30]:
data=[{'title':list(cut(item['title'])),'content':list(cut(item['content']))} for item in data]

In [4]:
len(data)

33876

In [31]:
with open(r'C:\Users\wie\Documents\项目\文本摘要\data\sample.json','w',encoding='utf-8') as f:
    json.dump(data,f,ensure_ascii=False)

In [5]:
from jieba import cut

In [5]:
def read_data(path=r'C:\Users\wie\Documents\项目\文本摘要\data\sample.json'):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    return ([item['title'] for item in data][:20000], [item['content'] for item in data][:20000]), \
           ([item['title'] for item in data][20000:], [item['content'] for item in data][20000:])

In [6]:
(train,l1),(test,l2)=read_data()

In [7]:
sorted(map(len,l1),reverse=True)[1000]            

551

In [8]:
sorted(map(len,l2),reverse=True)[:15]

[600, 600, 600, 600, 600, 600, 600, 600, 600, 600, 599, 599, 599, 599, 599]

In [9]:
sorted(map(len,[item['title'] for item in data]),reverse=True)[500]

37

In [10]:
sorted(map(len,[item['content'] for item in data]),reverse=True)[12000]

342

In [3]:
from jieba import cut

In [11]:
from datasets import load_metric
metric=load_metric(r'C:\Users\wie\Documents\项目\metrics\rouge.py')

In [4]:
model_path=r'C:\Users\wie\Documents\项目\model_file\summary'
model = MT5ForConditionalGeneration.from_pretrained(model_path)
# tokenizer实例化
tokenizer = T5PegasusTokenizer.from_pretrained(model_path)

In [7]:
len(list(cut(text)))

657

In [54]:
text="人民网哈尔滨8月13日电(王智圆、陈静)8月13日15时左右,在哈尔滨市香坊区延福街365路公交车站点,有人从一对老年夫妇手中抢走一名7岁男孩,并迅速驾驶一辆黑B牌照黑色轿车驶离现场。记者赶到事发辖区通天街派出所,目前从警方了解到的情况来看,可能涉及家庭纠纷。被抢走的孩子父亲去世,被判给母亲抚养,但孩子的爷爷奶奶并不想放弃孩子的抚养权,双方有一定的家庭矛盾。哈尔滨警方目前已经对各相关路段进行布控,并积极联系孩子母亲,找寻孩子下落尽快核实相关情况。事发路段哈尔滨市香坊区延福街"
input_ids=tokenizer(list(cut(text)),is_split_into_words=True,max_length=450,truncation=True,padding=True,return_tensors='pt')['input_ids']
out=model.generate(input_ids,max_length=60,num_beams=1)

In [55]:
tokenizer.decode(out[0],skip_special_tokens=True).replace(' ','')

'佛山男子拒男友分手,男友求复合不成后将其捅死。'

In [17]:
len(list(cut(text)))

133

In [14]:
text="人民网哈尔滨8月13日电(王智圆、陈静)8月13日15时左右,在哈尔滨市香坊区延福街365路公交车站点,有人从一对老年夫妇手中抢走一名7岁男孩,并迅速驾驶一辆黑B牌照黑色轿车驶离现场。记者赶到事发辖区通天街派出所,目前从警方了解到的情况来看,可能涉及家庭纠纷。被抢走的孩子父亲去世,被判给母亲抚养,但孩子的爷爷奶奶并不想放弃孩子的抚养权,双方有一定的家庭矛盾。哈尔滨警方目前已经对各相关路段进行布控,并积极联系孩子母亲,找寻孩子下落尽快核实相关情况。事发路段哈尔滨市香坊区延福街"